In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
# In this notebook, we try to fit a model of growth constant.
# The dataset consists of bended rectangles, done by applying an implicit module of order 1 with known linear growth law.
# To generate the dataset, simply execute "python3 generate_bending.py bendings.pickle nb_of_elem_in_dataset".
# We then fit a polynomial model of growth constants of order 2.

In [3]:
# Load our data
data = []
with open("bendings2.pickle", 'rb') as f:
    data = pickle.load(f)

template = data[0][0][:-1]
template = torch.unique_consecutive(template, dim=0)


true_implicit1_points = data[0][1]
true_C = data[0][2]
bendings = data[1:]

# Make sure each points in each data element are consecutively unique (needed when dealing varifold attachment atm).
dataset = [torch.unique_consecutive(target, dim=0)[:-1]  for target in list(zip(*bendings))[0]]
dataset = [target + 0.*torch.rand(target.shape) for target in dataset]
targets = dataset[:15]

print("Dataset size: {size}".format(size=len(targets)))

Dataset size: 10


In [4]:
%matplotlib qt5

# Display our data set and template
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
for target in targets:
    plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), color='grey', lw=2)
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
plt.plot(true_implicit1_points[:, 0].numpy(), true_implicit1_points[:, 1].numpy(), 'x')
plt.axis('equal')
plt.show()

In [5]:
# Defines the implicit module of order 1 that will be used by the atlas.
sigma_implicit1 = 1.5

implicit1_points = true_implicit1_points
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)
C_init = torch.ones(implicit1_points.shape[0], 2, 1)
implicit1 = dm.DeformationModules.ImplicitModule1(2, true_implicit1_points.shape[0], sigma_implicit1, C_init, nu=0.1, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()))


In [12]:
# Learning model: polynomial of order 2
# The polynomial model we will try to fit on our deformation constants
def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

# Callback called by the atlas. Serves as glue for our model of deformation constants.
def callback_compute_c_2(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    modules[1]._ImplicitModule1Base__C = pol_order_2(implicit1_points, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)

# Initial coefficient for the polynomial model of the deformation constants
abc_init = torch.zeros(6, 2)
abc_init[0] = torch.ones(2)    

In [6]:
# Learning model: polynomial of order 3 
# The polynomial model we will try to fit on our deformation constants
def pol_order_3(pos, a, b, c, d, e, f, g, h, i, j):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1] + g*pos[:, 0]**3 + h*(pos[:, 0]**2)*pos[:, 1] + i*pos[:, 0]*(pos[:, 1]**2) + j*pos[:, 1]**3

# Callback called by the atlas. Serves as glue for our model of deformation constants.
def callback_compute_c_3(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    g = abc[6].unsqueeze(1)
    h = abc[7].unsqueeze(1)
    i = abc[8].unsqueeze(1)
    j = abc[9].unsqueeze(1)
    modules[1]._ImplicitModule1Base__C = pol_order_3(implicit1_points, a, b, c, d, e, f, g, h, i, j).transpose(0, 1).unsqueeze(2)

    
# Initial coefficient for the polynomial model of the deformation constants
abc_init = torch.zeros(10, 2)
abc_init[0] = torch.ones(2)

In [7]:
Translation = dm.DeformationModules.GlobalTranslation(2, 1.)

In [8]:
# Creation of the atlas. Uses L2 norm attachment for now.
atlas = dm.Models.Atlas(template, [implicit1, Translation], [dm.Attachment.L2NormAttachment()], len(targets), lam=10000., other_parameters=[abc_init.clone().requires_grad_()], model_precompute_callback=callback_compute_c_3)
#atlas = dm.Models.Atlas(template, [implicit1], [dm.Attachment.VarifoldAttachment(2, [0.5, 2.])], len(targets), lam=10000., other_parameters=[abc_init.clone().requires_grad_()], model_precompute_callback=callback_compute_c_3)

In [9]:
# Fit the atlas
fitter = dm.Models.ModelFittingScipy(atlas)
costs = fitter.fit(targets, 150, options={'shoot_method': 'rk4', 'shoot_it': 10})

Initial energy = 2370489.070
{'attach_cost': 2370489.0703125, 'deformation_cost': 0.0, 'cost': 2370489.0703125}
Time: 136.12361037799928
Iteration: 1 
Total energy = 2181582.84765625 
Attach cost = 2181579.32421875 
Deformation cost = 3.505963975097984
Time: 183.7790713869981
Iteration: 2 
Total energy = 2105667.248046875 
Attach cost = 2105663.4609375 
Deformation cost = 3.7804584725527093
Time: 231.29396610999902
Iteration: 3 
Total energy = 2086397.466796875 
Attach cost = 2086394.087890625 
Deformation cost = 3.443153375061229
Time: 279.03140526899733
Iteration: 4 
Total energy = 2073343.90625 
Attach cost = 2073339.921875 
Deformation cost = 3.936607740353793
Time: 328.25704362800025
Iteration: 5 
Total energy = 2048307.412109375 
Attach cost = 2048301.09375 
Deformation cost = 6.28924420196563
Time: 377.64835236999716
Iteration: 6 
Total energy = 2033068.1875 
Attach cost = 2033060.39453125 
Deformation cost = 7.8248186856508255
Time: 427.0968876019979
Iteration: 7 
Total energy 

Time: 1949.6368445689986
Iteration: 38 
Total energy = 1286787.953125 
Attach cost = 1286399.61328125 
Deformation cost = 388.3291808962822
Time: 1999.7299810789991
Iteration: 39 
Total energy = 1282061.625 
Attach cost = 1281693.2265625 
Deformation cost = 368.3893079161644
Time: 2049.807411832997
Iteration: 40 
Total energy = 1278720.73046875 
Attach cost = 1278349.15234375 
Deformation cost = 371.5708293914795
Time: 2100.012282661999
Iteration: 41 
Total energy = 1275356.140625 
Attach cost = 1274967.87109375 
Deformation cost = 388.25184792280197
Time: 2149.1865975559995
Iteration: 42 
Total energy = 1268382.875 
Attach cost = 1267977.4140625 
Deformation cost = 405.44667422771454
Time: 2198.8301055449992
Iteration: 43 
Total energy = 1256133.87109375 
Attach cost = 1255712.494140625 
Deformation cost = 421.39593386650085
Time: 2248.1757424869975
Iteration: 44 
Total energy = 1228970.640625 
Attach cost = 1228518.537109375 
Deformation cost = 452.1225099861622
Time: 2297.50076101
I

Time: 3824.247181368999
Iteration: 76 
Total energy = 649447.0390625 
Attach cost = 648385.44921875 
Deformation cost = 1061.5942492485046
Time: 3873.5754511549967
Iteration: 77 
Total energy = 644842.40625 
Attach cost = 643754.44140625 
Deformation cost = 1087.9809947013855
Time: 3923.006128535999
Iteration: 78 
Total energy = 643715.599609375 
Attach cost = 642639.638671875 
Deformation cost = 1075.9585709571838
Time: 3971.8323053919994
Iteration: 79 
Total energy = 635816.58203125 
Attach cost = 634717.76953125 
Deformation cost = 1098.808705329895
Time: 4020.303588996998
Iteration: 80 
Total energy = 632560.646484375 
Attach cost = 631492.279296875 
Deformation cost = 1068.3707132339478
Time: 4069.3430368909976
Iteration: 81 
Total energy = 627696.49609375 
Attach cost = 626645.400390625 
Deformation cost = 1051.0860705375671
Time: 4117.275076807997
Iteration: 82 
Total energy = 618162.2353515625 
Attach cost = 617183.15625 
Deformation cost = 979.0632510185242
Time: 4217.42540528

Time: 5983.823516583998
Iteration: 113 
Total energy = 338380.283203125 
Attach cost = 337232.17822265625 
Deformation cost = 1148.099432528019
Time: 6032.042906970997
Iteration: 114 
Total energy = 333659.93896484375 
Attach cost = 332513.744140625 
Deformation cost = 1146.192772448063


KeyboardInterrupt: 

In [ ]:
plt.plot(range(len(costs)), costs)
plt.show()

In [10]:
# Retrieve the optimised coeffcient and then compare the generated growth constraint to the true one.
abc_fit = atlas.parameters[-1].detach()
print(0.1*abc_fit)
print(torch.tensor([[0., 0.3], [0., 1.], [0., 0.], [0., 0.], [0., 1.], [0., -1.] ]))

C_fit = pol_order_3(implicit1_points, abc_fit[0].unsqueeze(1), abc_fit[1].unsqueeze(1), abc_fit[2].unsqueeze(1), abc_fit[3].unsqueeze(1), abc_fit[4].unsqueeze(1), abc_fit[5].unsqueeze(1), abc_fit[6].unsqueeze(1), abc_fit[7].unsqueeze(1), abc_fit[8].unsqueeze(1), abc_fit[9].unsqueeze(1)).t().unsqueeze(2)

# The closer to 1. it is, the better.
print(torch.dot((true_C/torch.norm(true_C)).flatten(), (C_fit/torch.norm(C_fit)).flatten()))

tensor([[-0.0348,  0.4127],
        [ 0.0221,  0.4396],
        [-0.1430,  0.0342],
        [-0.1232,  0.0267],
        [-0.1537,  0.5526],
        [ 0.2183, -0.5118],
        [ 0.0047,  0.1092],
        [-0.0045,  0.0137],
        [ 0.0212, -0.1132],
        [-0.0379,  0.0055]])
tensor([[ 0.0000,  0.3000],
        [ 0.0000,  1.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  1.0000],
        [ 0.0000, -1.0000]])
tensor(0.9394)


In [11]:
ax0 = plt.subplot(111, aspect='equal')
dm.Utilities.plot_C_arrow(ax0, implicit1_points, true_C, R=implicit1_R, scale=0.05, zorder=3, mutation_scale=10)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, true_C, R=R, scale=1., color='black')

In [12]:
pts = atlas.models[0].init_manifold.gd[1][0].detach()
R = atlas.models[0].init_manifold.gd[1][1].detach()
ax1 = plt.subplot(111, aspect='equal')

dm.Utilities.plot_C_arrow(ax1, pts, C_fit, R=R, scale=0.05, zorder=3, mutation_scale=10)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C_fit, R=R, scale=1., color='black')

/home/gris/project_py3/lib/python3.5/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [11]:
source = atlas.models[0].init_manifold.gd[0]

In [12]:
i=7
for i in range(10):
    shoot_method = 'rk4'
    shoot_it = 10
    silent_pos = atlas.models[i].init_manifold[0].gd.detach().clone()
    silent_mom = atlas.models[i].init_manifold[0].cotan.detach().clone()
    silent = dm.DeformationModules.SilentLandmarks(2, silent_pos.shape[0], gd=silent_pos, cotan=silent_mom)

    pts = atlas.models[i].init_manifold.gd[1][0].detach().clone()
    R = atlas.models[i].init_manifold.gd[1][1].detach().clone()
    imp1_cotan_pts = atlas.models[i].init_manifold.cotan[1][0].detach().clone()
    imp1_cotan_R = atlas.models[i].init_manifold.cotan[1][1].detach().clone()
    implicit1 = dm.DeformationModules.ImplicitModule1(2, pts.shape[0], sigma_implicit1, C_fit, nu=0.1, gd=(pts.clone().requires_grad_(), R.clone().requires_grad_()), cotan=(imp1_cotan_pts.clone().requires_grad_(), imp1_cotan_R.clone().requires_grad_()))


    h = dm.HamiltonianDynamic.Hamiltonian([silent, implicit1])
    intermediate_states, _ = dm.HamiltonianDynamic.shoot(h, shoot_it, shoot_method, intermediates=True)

    plt.figure()
    plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
    target = targets[i].detach().clone()

    t = -1
    transported_source = intermediate_states[t].gd[0].detach()
    plt.plot(transported_source[:, 0].numpy(), transported_source[:, 1].numpy(), color='red')
    plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), '--')

    plt.axis('equal')

In [60]:
plt.close('all')

In [114]:

points_per_side = 40

silent_points = dm.Utilities.generate_unit_square(points_per_side)
silent_points = silent_points*torch.tensor([1., 4.])
aabb = dm.Utilities.AABB.build_from_points(silent_points).scale(1.2)
implicit1_points = aabb.fill_uniform_density(20.)
implicit1_points = implicit1_points - torch.mean(implicit1_points, dim=0)
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)

def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

abc = torch.tensor([[0., 0.3], [0., 1.], [0., 0.], [0., 0.], [0., 1.], [0., -1.] ])
a = abc[0].unsqueeze(1)
b = abc[1].unsqueeze(1)
c = abc[2].unsqueeze(1)
d = abc[3].unsqueeze(1)
e = abc[4].unsqueeze(1)
f = abc[5].unsqueeze(1)
C = pol_order_2(implicit1_points, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)

#C = torch.zeros(implicit1_points.shape[0], 2, 1)
#C[:, 1, 0] = 10.*torch.abs(implicit1_points[:, 0] - torch.mean(implicit1_points, dim=0)[0] + 0.5*aabb.width)/aabb.width

ax1 = plt.subplot(111, aspect='equal')
plt.plot(silent_points[:,0], silent_points[:,1], '.b')
plt.plot(implicit1_points[:,0], implicit1_points[:,1], '.r')

dm.Utilities.plot_C_arrow(ax1, implicit1_points, C, R=implicit1_R, scale=1., zorder=3, mutation_scale=10)


In [115]:

def generate_bending():
    def _generate():
        silent_cotan = torch.randn_like(silent_points)
        implicit1_cotan = torch.zeros_like(implicit1_points)
        #val = 1000.*torch.rand(1) - 500.
        # val = 100.*torch.randn(1)

        # implicit1_cotan[0::7, 0] = val
        # implicit1_cotan[6::7, 0] = val
        #implicit1_cotan[6, 0] = val
        #implicit1_cotan[-1, 0] = 
        implicit1_cotan = 15.*torch.randn_like(implicit1_points)
        implicit1_cotan_R = torch.zeros_like(implicit1_R)

        # plt.plot(silent_points[:, 0].numpy(), silent_points[:, 1].numpy())
        # plt.quiver(implicit1_points[:, 0].numpy(), implicit1_points[:, 1].numpy(), implicit1_cotan[:, 0].numpy(), implicit1_cotan[:, 1].numpy())
        # plt.axis('equal')
        # plt.show()

        # ax = plt.subplot()
        # plt.plot(silent_points[:, 0].numpy(), silent_points[:, 1].numpy())
        # dm.Utilities.plot_C_arrow(ax, implicit1_points, C, scale=0.1, mutation_scale=10.)
        # plt.axis('equal')
        # plt.show()

        silent = dm.DeformationModules.SilentLandmarks(2, silent_points.shape[0], gd=silent_points.clone().requires_grad_(), cotan=silent_cotan.clone().requires_grad_())

        implicit1 = dm.DeformationModules.ImplicitModule1(2, implicit1_points.shape[0], 0.5, C, nu=0.1, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()), cotan=(implicit1_cotan.clone().requires_grad_(), implicit1_cotan_R.clone().requires_grad_()))

        with torch.autograd.no_grad():
            dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, implicit1]), 10, 'rk4')

        return silent.manifold.gd.detach(), implicit1.manifold.gd[0].detach()

    while True:
        bending = _generate()
        #print(1.5*torch.norm(silent_points), " ", torch.norm(bending[0]), " ", 4.*torch.norm(silent_points))
        if (not torch.any(torch.isnan(bending[0]))) and (torch.norm(bending[0]) >= 0.*torch.norm(silent_points)) and (torch.norm(bending[0]) <= 100.*torch.norm(silent_points)):
            print("New data element added. Frobenius norm: {norm}".format(norm=torch.norm(bending[0])))
            return bending


In [118]:
tar = generate_bending()

New data element added. Frobenius norm: 23.770769119262695


In [119]:

ax1 = plt.subplot(111, aspect='equal')
plt.plot(silent_points[:,0], silent_points[:,1], '.b')
plt.plot(implicit1_points[:,0], implicit1_points[:,1], '.r')

plt.plot(tar[0][:,0], tar[0][:,1], '-g')


tensor(21.4277)